# This currently is a dumpster for an attempt to transform the 3d files. It is not working code.

In [ ]:
from collections.abc import Iterable
import xarray as xr
import numpy as np

In [ ]:
def hacky_linear_interpolation(
    data, data_grid, target, dim="level_full", data_grid_order="descending"
):
    def adjust_dims(dims):
        return ["pressure"] + [x for x in list(ds_new["ms_dh"].dims) if x != "lev"]

    def arrayfrom(data, template: xr.DataArray):
        return xr.DataArray(
            data,
            dims=adjust_dims(template.dims),
            name=template.name,
            attrs=template.attrs,
        ).transpose("time", "pressure", "cells")

    if isinstance(target, Iterable) and len(target) > 1:
        return arrayfrom(
            [hacky_linear_interpolation(data, data_grid, y, dim) for y in target], data
        )
    else:
        if data_grid_order == "descending":
            level_above = (data_grid < target).argmax(dim=dim)
        else:
            level_above = (data_grid > target).argmax(dim=dim)
        level_below = level_above - 1
        value_above = data_grid.isel(**{dim: level_above})
        value_below = data_grid.isel(**{dim: level_below})
        f = (target - value_below) / (value_above - value_below)
        interpolated = (1 - f) * data.isel(**{dim: level_below}) + f * data.isel(
            **{dim: level_above}
        )
        return xr.where(level_above > 0, interpolated, np.nan)

In [ ]:
ds_new["zg"] = (
    ds_new["lev"]
    .expand_dims(
        dim={x: len(ds_new[x]) for x in ds_new["ms_dh"].dims if not x == "lev"}
    )
    .transpose("time", "lev", "cells")
)

In [ ]:
tr = np.arange(100, 900, 100)
lt = np.arange(850, 1025, 25)
ua = np.arange(10, 90, 20)
levels = sorted({1, 5, 20, 150, 250, 750}.union(tr, lt, ua))
pressure_levels = xr.DataArray(
    levels,
    dims="pressure",
    name="pressure",
)
pressure_levels

<xarray.DataArray 'pressure' (pressure: 25)> Size: 200B
array([   1,    5,   10,   20,   30,   50,   70,  100,  150,  200,  250,
        300,  400,  500,  600,  700,  750,  800,  850,  875,  900,  925,
        950,  975, 1000])
Dimensions without coordinates: pressure

In [ ]:
xr.Dataset(
    {
        var: hacky_linear_interpolation(
            ds_new[var], ds_new["ms_pres"], pressure_levels.values, dim="lev"
        )
        for var in ds_new
        if "lev" in ds_new[var].dims
    }
    | {"pressure": pressure_levels}
)

<xarray.Dataset> Size: 2MB
Dimensions:   (time: 44, pressure: 25, cells: 12)
Coordinates:
  * pressure  (pressure) int64 200B 1 5 10 20 30 50 ... 875 900 925 950 975 1000
Dimensions without coordinates: time, cells
Data variables: (12/16)
    cli       (time, pressure, cells) float64 106kB nan nan nan ... 0.0 0.0 0.0
    clw       (time, pressure, cells) float64 106kB nan nan nan ... 0.0 0.0 0.0
    hur       (time, pressure, cells) float64 106kB nan nan ... 0.0008212
    hus       (time, pressure, cells) float64 106kB nan nan ... 5.194e-06
    ms_dh     (time, pressure, cells) float64 106kB nan nan ... 2.994e-07
    ms_lwhr   (time, pressure, cells) float64 106kB nan nan ... -4.76e-05
    ...        ...
    qs        (time, pressure, cells) float64 106kB nan nan nan ... 0.0 0.0 0.0
    ta        (time, pressure, cells) float64 106kB nan nan nan ... 231.1 232.4
    ua        (time, pressure, cells) float64 106kB nan nan ... -11.06 -11.77
    va        (time, pressure, cells) float64 106kB nan nan nan ... 0.8049 1.345
    wa        (time, pressure, cells) float64 106kB nan nan ... -0.0004156
    zg        (time, pressure, cells) float64 106kB nan nan ... 3.12e+04

In [1]:
def process_3d(curr_conf, out_ds):
    out_ds["zg"] = (
        out_ds["lev"]
        .expand_dims(
            dim={x: len(out_ds[x]) for x in out_ds["ms_dh"].dims if not x == "lev"}
        )
        .transpose("time", "lev", "cells")
    )
    # out_ds = hacky_linear_interpolation(out_ds, out_ds['ms_pres'], target, dim="lev")
    return out_ds

In [2]:
def convert_files(zoom, subset):
    files = (
        f"/large/sftpgo/data/NICAM/hackathon/tksk_sample/*/{subset}/z{zoom:02d}/*.nc"
    )
    outfile = f"{output_dir}/NICAM_{subset}_z{zoom}.zarr"
    curr_conf = config[subset]
    timechunk = curr_conf["chunks"]["time"]

    ds = xr.open_mfdataset(files, chunks=curr_conf['chunks'])
    out_ds = process_3d(curr_conf, ds)

    ! rm -rf {outfile}
    to_zarr_store(path=outfile, outds=out_ds, timechunk=timechunk, order=zoom)
    return outfile

In [3]:
def to_zarr_store(path, outds, timechunk, order):
    store = zarr.storage.DirectoryStore(
        path, normalize_keys=False, dimension_separator="/"
    )
    outds.to_zarr(
        store, encoding=chunk_tools.get_encodings(outds=outds, timechunk=timechunk, order=order)
    )  # , compute=False
    store.close()